In [131]:
import os
from pathlib import Path
import glob
import tensorflow as tf
from tensorflow.core.util import event_pb2
from tensorboard.plugins.hparams import plugin_data_pb2
import inspect

In [36]:
os.chdir(r"C:\Users\robin\Documents\projets\pokemon_dataset")

In [37]:
# Get all the tensorboard files of all runs
runs_folders = glob.glob("runs/*/", recursive=False)
run_folder = runs_folders[-1]
# Choose the last run
os.chdir(run_folder)

In [38]:
events_files = glob.glob('**/*.v2',recursive=True)

In [39]:
events_files

['events.out.tfevents.1640250140.MSI.8052.2.v2',
 'events.out.tfevents.1640250146.MSI.8052.3.v2',
 'events.out.tfevents.1640251623.MSI.8052.6.v2',
 'train\\events.out.tfevents.1640250140.MSI.8052.0.v2',
 'train\\events.out.tfevents.1640250146.MSI.8052.4.v2',
 'validation\\events.out.tfevents.1640250140.MSI.8052.1.v2',
 'validation\\events.out.tfevents.1640250251.MSI.8052.5.v2']

There are 3 categories of summaries : 
- metrics (scalar values) (x2 for tr val)
- text infos on the run (x2 for tr val) : index 3
- final_accuracy metric
- hparams -> structure : index 0 ; values chosen : index 1 et 2

### Metrics

In train and valid folders:
⚠️ contains keras graph

accumulate in a buffer and compute intervals
⚠️ infinity metrics : write inf in this case 

In [60]:
summary = tf.data.TFRecordDataset(events_files[4])
for serialized_example in summary:
    event = event_pb2.Event.FromString(serialized_example.numpy())
    for value in event.summary.value:
        t = tf.make_ndarray(value.tensor)
        print(value.tag, t)

keras b'{"class_name": "Functional", "config": {"name": "model", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 256, 256, 3], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_1"}, "name": "input_1", "inbound_nodes": []}, {"class_name": "Conv2D", "config": {"name": "conv2d", "trainable": true, "dtype": {"class_name": "Policy", "config": {"name": "mixed_float16"}, "shared_object_id": 4}, "filters": 3, "kernel_size": [3, 3], "strides": [1, 1], "padding": "same", "data_format": "channels_last", "dilation_rate": [1, 1], "groups": 1, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": {"class_name": "L1L2", "config": {"l1": 9.99999993922529e-09, "l2": 9.99999993922529e-09}}, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, "name": "

### Text

Identifiable par value.tag = Description

Action : copier coller

In [196]:
summary = tf.data.TFRecordDataset(events_files[3])
for serialized_example in summary:
    event = event_pb2.Event.FromString(serialized_example.numpy())
    for value in event.summary.value:
        t = tf.make_ndarray(value.tensor)
        print(value.tag, str(t))
        v = str(t)

Description commit b'be58d26 batchnorm tests, 64,128,256 sans batchnorm'
b'be58d26 batchnorm tests, 64,128,256 sans batchnorm'


### HParams

Structure détectable lorsque value.tag contient _hparams_ :
- partie uniquement structure contient experiment
- partie paramètres choisis contient session_start_info (x2)

Action à effectuer: copier coller les fichiers directement

In [115]:

summary = tf.data.TFRecordDataset(events_files[0])
for serialized_example in summary:
    event = event_pb2.Event.FromString(serialized_example.numpy())
    for value in event.summary.value:
        t = tf.make_ndarray(value.tensor)
        print(value.tag, t)
        print(plugin_data_pb2.HParamsPluginData.FromString(value.metadata.plugin_data.content))
        v1 = plugin_data_pb2.HParamsPluginData.FromString(value.metadata.plugin_data.content) 
        v = v1 if str(v1) != "" else v

_hparams_/experiment []
experiment {
  hparam_infos {
    name: "batch_size"
    type: DATA_TYPE_FLOAT64
    domain_interval {
      min_value: 1.0
      max_value: 512.0
    }
  }
  hparam_infos {
    name: "learning_rate_1e-"
    type: DATA_TYPE_FLOAT64
    domain_interval {
      max_value: 9.0
    }
  }
  hparam_infos {
    name: "filter1_2**"
    type: DATA_TYPE_FLOAT64
    domain_interval {
      max_value: 20.0
    }
  }
  hparam_infos {
    name: "filter2_2**"
    type: DATA_TYPE_FLOAT64
    domain_interval {
      max_value: 20.0
    }
  }
  hparam_infos {
    name: "filter3_2**"
    type: DATA_TYPE_FLOAT64
    domain_interval {
      max_value: 20.0
    }
  }
  hparam_infos {
    name: "filter4_2**"
    type: DATA_TYPE_FLOAT64
    domain_interval {
      max_value: 20.0
    }
  }
  hparam_infos {
    name: "filter5_2**"
    type: DATA_TYPE_FLOAT64
    domain_interval {
      max_value: 20.0
    }
  }
  hparam_infos {
    name: "filter6_2**"
    type: DATA_TYPE_FLOAT64
    dom

In [116]:
print(type(v))

<class 'tensorboard.plugins.hparams.plugin_data_pb2.HParamsPluginData'>


In [183]:
values = [v.experiment.hparam_infos[i] for i in range(len(v.experiment.hparam_infos))]
print((values))

[name: "batch_size"
type: DATA_TYPE_FLOAT64
domain_interval {
  min_value: 1.0
  max_value: 512.0
}
, name: "learning_rate_1e-"
type: DATA_TYPE_FLOAT64
domain_interval {
  max_value: 9.0
}
, name: "filter1_2**"
type: DATA_TYPE_FLOAT64
domain_interval {
  max_value: 20.0
}
, name: "filter2_2**"
type: DATA_TYPE_FLOAT64
domain_interval {
  max_value: 20.0
}
, name: "filter3_2**"
type: DATA_TYPE_FLOAT64
domain_interval {
  max_value: 20.0
}
, name: "filter4_2**"
type: DATA_TYPE_FLOAT64
domain_interval {
  max_value: 20.0
}
, name: "filter5_2**"
type: DATA_TYPE_FLOAT64
domain_interval {
  max_value: 20.0
}
, name: "filter6_2**"
type: DATA_TYPE_FLOAT64
domain_interval {
  max_value: 20.0
}
, name: "filter7_2**"
type: DATA_TYPE_FLOAT64
domain_interval {
  max_value: 20.0
}
, name: "middle_filter_2**"
type: DATA_TYPE_FLOAT64
domain_interval {
  max_value: 20.0
}
, name: "u_skip"
type: DATA_TYPE_BOOL
domain_discrete {
  values {
    bool_value: false
  }
  values {
    bool_value: true
  }
}
, 

In [126]:
print((list(v.session_start_info.hparams.items())[0][1]).items()  )

IndexError: list index out of range